In [1]:
import numpy as np
import pandas as pd
from possibilearn import *
from possibilearn.kernel import GaussianKernel
from clustering import *
import math
import random

#sample generator
def g(m):
    return (-4 + np.random.random(2*m) * 8).reshape((m, 2))

def get_different_clusters(clusters, clusters_index, clusters_labels):
    d={}
    c=[]
    l=[]
    i=[]
    for j in range(len(clusters_labels)):
        if clusters_labels[j] not in d:
            d[clusters_labels[j]] = True
            c.append(clusters[j])
            l.append(clusters_labels[j])
            i.append(clusters_index[j])
    return c, i, l
        

source = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

iris_df = pd.read_csv(
    filepath_or_buffer=source,
    header=None,
    sep=',')

iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # drops the empty line at file-end

iris_values = iris_df.iloc[:,:4].values
iris_labels = iris_df.iloc[:,4].values 

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

iris_values_std = StandardScaler().fit_transform(iris_values)

pca_2d = PCA(n_components=2)
iris_values_2d = pca_2d.fit_transform(iris_values_std)

In [2]:
sigmas = np.array([0.288, 0.250, 0.222])
cs = np.array([10**i for i in range(-2,3)])
couples = [(c,s) for s in sigmas for c in cs]

In [3]:
n = len(iris_values_2d)
iterations = 10

validation_accuracy={}
test_accuracy={}
training_accuracy={}
valid_iteration = 0
while(valid_iteration < iterations):
    print '\n\nholdout iteration {}'.format(valid_iteration)
    
    #random permutation of train validation and test sets
    permutation = np.random.permutation(range(n))
    perc_train, perc_val, perc_test = (.8, .1, .1)
    train_index = permutation[:int(n*perc_train)]
    validation_index = permutation[int(n*perc_train):int(n*(perc_train+perc_val))]
    test_index = permutation[int(n*(perc_train+perc_val)):]
    train_set = iris_values_2d[[i for i in train_index]]
    validation_set = iris_values_2d[[i for i in validation_index]]
    test_set = iris_values_2d[[i for i in test_index]]
    
    # find the couple that have the best accuracy for this permutation
    best_couple = (-1,-1) #initialization of best couple
    best_accuracy = -1 #initialization of best accuracy
    
    for (c, sigma) in couples:
        print '\nmodel selection: trying parameters c={}, sigma={}'.format(c, sigma)
        print 'starting clusterization'
        try:
            clusters_index, r = clustering(train_set, sigma, c)
            clusters_index = [cl for cl in clusters_index if len(cl) > 1] #exclude singleton clusters
            
            #sort clusters 
            clusters_index.sort(key = lambda x: -len(x))
            
            #clusters_index = clusters_index[:3]
            
            clusters = [train_set[cl] for cl in clusters_index] #get the clusters with the points
            
            clusters_labels = [pd.Series([iris_labels[train_index[i]] for i in clusters_index[k]])
                          .mode()[0] for k in range(len(clusters))] #associate clusters with labels
            print 'first labels: ', clusters_labels
            
            clusters, clusters_index, clusters_labels = get_different_clusters(clusters, clusters_index, clusters_labels)
            
            print 'second labels: ',clusters_labels   
                
                
            #the clusters must be 3 and represent the 3 different labels
            if len(clusters) == 3 and len(set(clusters_labels)) == 3:
                print 'got enough clusters'
                print 'labelled clusters ', clusters_labels
                
                cluster_indices = [clusters_index[cluster_id] for cluster_id in range(3)]
                mus = [[1 if i in cc else 0 for i in range(int(n*perc_train))] for cc in cluster_indices]
                
                estimated_memberships = []
                for i in range(3):
                    print 'inferring membership {}'.format(i)
                    estimated_membership, _ = possibility_learn(train_set,
                                                      mus[i],
                                                      c=c,
                                                      k=GaussianKernel(sigma),
                                                      sample_generator=g)
                    estimated_memberships.append(estimated_membership)
                
                if not (None in estimated_memberships):
                    
                    print 'validating model'

                    guessed_labels = np.array([clusters_labels[np.argmax([e(t) for e in estimated_memberships])] 
                                       for t in validation_set])

                    correct_labels = np.array(iris_labels[validation_index])
                    print 'correct labels: ', correct_labels
                    print 'guessed labels: ', guessed_labels
                    accuracy = float(sum(guessed_labels == correct_labels)) / len(validation_set)
                    print 'accuracy', accuracy
                    
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_couple = c, sigma
                    
                else:
                    print 'unable to infer an estimated membership function'
                    continue
                            
            else:
                print 'not enough clusters'
                continue
            
        except ValueError as e:
            print str(e)
            
    c_best, sigma_best = best_couple
    if c_best > 0 and sigma_best > 0:
        
        #with the couple with the best accuracy infer the membership function merging train and validation set
        new_train_set = np.vstack((train_set, validation_set))
        new_train_index = np.hstack((train_index, validation_index))
        
        print '\nstarting training with best couple: ', best_couple 
        print 'starting clusterization'
        
        try:
            clusters_index, r = clustering(new_train_set, sigma, c)
            clusters_index = [cl for cl in clusters_index if len(cl) > 1] #exclude singleton clusters
            
            #sort clusters 
            clusters_index.sort(key = lambda x: -len(x))
            
            #clusters_index = clusters_index[:3]
            clusters = [new_train_set[cl] for cl in clusters_index] #get the clusters with the points
            
            clusters_labels = [pd.Series([iris_labels[new_train_index[i]] for i in clusters_index[k]])
                          .mode()[0] for k in range(len(clusters))] #associate clusters with labels
            
            clusters, clusters_index, clusters_labels = get_different_clusters(clusters, clusters_index, clusters_labels)
                     
            #the clusters must be 3 and represent the 3 different labels
            if len(clusters) == 3 and len(set(clusters_labels)) == 3:
                print 'got enough clusters'
                print 'labelled clusters ', clusters_labels
                
                cluster_indices = [clusters_index[cluster_id] for cluster_id in range(3)]
                mus = [[1 if i in cc else 0 for i in range(int(n*(perc_train+perc_val)))] for cc in cluster_indices]
                
                estimated_memberships = []
                for i in range(3):
                    print 'inferring membership {}'.format(i)
                    estimated_membership, _ = possibility_learn(new_train_set,
                                                      mus[i],
                                                      c=c,
                                                      k=GaussianKernel(sigma),
                                                      sample_generator=g)
                    estimated_memberships.append(estimated_membership)
                
                if not (None in estimated_memberships):
                    
                    print 'validating model'
                    print 'the test is valid'
                    
                    #compute the accuracy on test set
                    guessed_labels = np.array([clusters_labels[np.argmax([e(t) for e in estimated_memberships])] 
                                       for t in test_set])

                    correct_labels = np.array(iris_labels[test_index])
                    print 'correct labels: ', correct_labels
                    print 'guessed labels: ', guessed_labels
                    accuracy = float(sum(guessed_labels == correct_labels)) / len(validation_set)
                    print 'accuracy of the best couple on test set is ', accuracy
                    
                    test_accuracy[(valid_iteration, c, sigma)] = accuracy
                    
                    #compute the accuracy on training set
                    guessed_labels = np.array([clusters_labels[np.argmax([e(t) for e in estimated_memberships])] 
                                       for t in new_train_set])

                    correct_labels = np.array(iris_labels[new_train_index])
                    print 'correct labels: ', correct_labels
                    print 'guessed labels: ', guessed_labels
                    accuracy = float(sum(guessed_labels == correct_labels)) / len(validation_set)
                    print 'accuracy of the best couple on training set is ', accuracy
                    
                    training_accuracy[(valid_iteration, c, sigma)] = accuracy
                    valid_iteration = valid_iteration +1
                    
                else:
                    print 'unable to infer an estimated membership function'
                    continue
                            
            else:
                print 'not enough clusters'
                continue
            
        except ValueError as e:
            print str(e)
    
    else:
        print'this permutation has not produced a best couple'
        
        
print 'accuracy for the valid tests', test_accuracy



holdout iteration 0

model selection: trying parameters c=0.01, sigma=0.288
starting clusterization
first labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
second labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
inferring membership 0
inferring membership 1
inferring membership 2
unable to infer an estimated membership function

model selection: trying parameters c=0.1, sigma=0.288
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa']
second labels:  ['Iris-versicolor', 'Iris-setosa']
not enough clusters

model selection: trying parameters c=1.0, sigma=0.288
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa']
second labels:  ['Iris-versicolor', 'Iris-setosa']
not enough clusters

model selection: trying parameters c=10.0, sigma=0.288
st

inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica'
 'Iris-setosa' 'Iris-setosa' 'Iris-virginica']
guessed labels:  ['Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor']
accuracy 0.866666666667

model selection: trying parameters c=100.0, sigma=0.222
starting clusterization
first labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica']
second labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']

first labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor']
second labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa']
guessed labels:  ['Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa']
accuracy 0.933333333333

model selection: trying parameters c=10.0, sigma=0.288
starting clusterization
first labels:  [

first labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor']
second labels:  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-virginica', 'Iris-versicolor']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa']
guessed labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa']
accuracy 0.866666666667

model selection: trying parameters c=10.0, sigma=0.222
starting clusteriza

first labels:  ['Iris-versicolor']
second labels:  ['Iris-versicolor']
not enough clusters

model selection: trying parameters c=0.1, sigma=0.288
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor']
second labels:  ['Iris-versicolor', 'Iris-setosa']
not enough clusters

model selection: trying parameters c=1.0, sigma=0.288
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor']
second labels:  ['Iris-versicolor', 'Iris-setosa']
not enough clusters

model selection: trying parameters c=10.0, sigma=0.288
starting clusterization
first labels:  ['Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor']
second labels:  ['Iris-virginica', 'Iris-setosa', 'Iris-versicolor']
got enough clusters
labelled clusters  ['Iris-virginica', 'Iris-setosa', 'Iris-versicolor']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-setosa'

...

inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa']
guessed labels:  ['Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa']
accuracy 0.733333333333

model selection: trying parameters c=0.01, sigma=0.222
starting clusterization
first labels:  ['Iris-setosa', 'Iris-versicolor', 'Iris-versicolor']
second labels:  ['Iris-setosa', 'Iris-versicolor']
not enough clusters

model selection: trying parameters c=0.1, sigma=0.222
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa',

first labels:  ['Iris-versicolor']
second labels:  ['Iris-versicolor']
not enough clusters

model selection: trying parameters c=0.1, sigma=0.288
starting clusterization
first labels:  ['Iris-setosa', 'Iris-versicolor']
second labels:  ['Iris-setosa', 'Iris-versicolor']
not enough clusters

model selection: trying parameters c=1.0, sigma=0.288
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa']
second labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
got enough clusters
labelled clusters  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor']
guessed labels:  ['Iris-versico

first labels:  ['Iris-versicolor', 'Iris-versicolor']
second labels:  ['Iris-versicolor']
not enough clusters

model selection: trying parameters c=10.0, sigma=0.222
starting clusterization
first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica']
second labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
got enough clusters
labelled clusters  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor']
guessed labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versic

first labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
second labels:  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
got enough clusters
labelled clusters  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor']
guessed labels:  ['Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor']
accuracy 0.666666666667

model selection: trying parameters c=10.0, sigma=0.288
starting clusterization
first labels:  ['Iris-versicolo

inferring membership 1
inferring membership 2
validating model
correct labels:  ['Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor']
guessed labels:  ['Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor']
accuracy 0.866666666667

model selection: trying parameters c=10.0, sigma=0.222
starting clusterization
first labels:  ['Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica']
second labels:  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
got enough clusters
labelled clusters  ['I

In [4]:
print 'accuracy mean for the training set: ', np.array(training_accuracy.values()).mean()
print 'accuracy std deviation for the training set: ', np.array(training_accuracy.values()).std()

print 'accuracy mean for the test set: ', np.array(test_accuracy.values()).mean()
print 'accuracy std deviation for the test set: ', np.array(test_accuracy.values()).std()

accuracy mean for the training set:  7.4
accuracy std deviation for the training set:  0.2812669747963865
accuracy mean for the test set:  0.8
accuracy std deviation for the test set:  0.06666666666666671
